### Data Collection of various useful parameters of more than 200 cities across the world

First, I have extracted a list of more than 200 cities having parameters like Quality of Life Index, Purchasing Power Index, Safety Index, Health Care Index, Cost of Living Index, Property Price to Income Ratio, Trafic Commute Time Index, Pollution Index & Climate Index through web scraping from the url - "https://www.numbeo.com/quality-of-life/rankings_current.jsp".

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Send a GET request to the URL
url = "https://www.numbeo.com/quality-of-life/rankings_current.jsp"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Find the table containing the data
table = soup.find("table", id="t2")

# Initialize an empty list to store the data
data = []

# Iterate through each row in the table body
for row in table.find("tbody").find_all("tr"):
    # Initialize an empty dictionary to store the row data
    row_data = {}
    
    # Extract data from each column in the row
    columns = row.find_all("td")
    row_data["City"] = columns[1].text.strip()
    row_data["Quality of Life Index"] = columns[2].text.strip()
    row_data["Purchasing Power Index"] = columns[3].text.strip()
    row_data["Safety Index"] = columns[4].text.strip()
    row_data["Health Care Index"] = columns[5].text.strip()
    row_data["Cost of Living Index"] = columns[6].text.strip()
    row_data["Property Price to Income Ratio"] = columns[7].text.strip()
    row_data["Traffic Commute Time Index"] = columns[8].text.strip()
    row_data["Pollution Index"] = columns[9].text.strip()
    row_data["Climate Index"] = columns[10].text.strip()
    
    # Append the row data to the list
    data.append(row_data)

# Define the CSV file name
csv_file = "quality_of_life_indices.csv"

# Write the scraped data to the CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ["City", "Quality of Life Index", "Purchasing Power Index", "Safety Index",
                  "Health Care Index", "Cost of Living Index", "Property Price to Income Ratio",
                  "Traffic Commute Time Index", "Pollution Index", "Climate Index"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header
    writer.writeheader()
    
    # Write the data
    writer.writerows(data)

print(f"Data has been written to {csv_file}")


Data has been written to quality_of_life_indices.csv


Some Data cleaning performed for City column to have uniform representation across the task.

Country column extracted from the City.

In [8]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('quality_of_life_indices.csv')

# Define a function to extract city and country
def extract_city_country(city_country):
    parts = city_country.split(',')
    if len(parts) == 1:
        return parts[0].strip(), ''
    elif len(parts) == 2:
        return parts[0].strip(), parts[1].strip()
    else:
        return parts[0].strip(), parts[-1].strip()

# Apply the function to the 'City' column and create new 'city' and 'country' columns
df['city'], df['country'] = zip(*df['City'].apply(extract_city_country))

# Drop the original 'City' column
df.drop(columns=['City'], inplace=True)

# Reorder columns with 'city' and 'country' at the beginning
df = df[['city', 'country'] + [col for col in df.columns if col != 'city' and col != 'country']]

# Save the modified DataFrame to a new CSV file
df.to_csv('modified_quality_of_life_indices.csv', index=False)


Collection of few more information for each cities like Population, Elevation & coordinates(useful for further extraction) through Geocoding API.

In [10]:
# Read data from modified_data.csv
df = pd.read_csv('modified_quality_of_life_indices.csv')

# Initialize lists to store city-wise information
cities = df.iloc[:, 0].tolist()  # Get cities from the first column

city_info = []

# Iterate over each city
for city in cities:
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1&language=en&format=json"
    response = requests.get(url)
    data = response.json()
    
    # Check if data is available for the city
    if 'results' in data and len(data['results']) > 0:
        result = data['results'][0]
        latitude = result['latitude']
        longitude = result['longitude']
        elevation = result['elevation'] if 'elevation' in result else None
        population = result['population'] if 'population' in result else None
        
        city_info.append({
            'City': city,
            'Latitude': latitude,
            'Longitude': longitude,
            'Elevation': elevation,
            'Population': population
        })
    else:
        print(f"No data found for {city}")

# Create a DataFrame from the collected city information
city_info_df = pd.DataFrame(city_info)

# Write the DataFrame to a new CSV file
city_info_df.to_csv('city_information.csv', index=False)

print("City information has been stored in city_information.csv")


City information has been stored in city_information.csv


Collection of weather parameters like Tempertaure, Humidity & Percipitation on monthly average basis through API.

Every columns contains 12 space seperated values for each months.

In [11]:
from datetime import datetime

# Function to extract monthly average weather data
def extract_monthly_weather(lat, lon):
    start_date = "2023-01-01"
    end_date = "2023-12-31"
    url = f"https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={lon}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,precipitation"
    response = requests.get(url)
    data = response.json()
    if 'hourly' in data and 'time' in data['hourly'] and 'temperature_2m' in data['hourly']:
        hourly_temperatures = data['hourly']['temperature_2m']
        hourly_humidity = data['hourly']['relative_humidity_2m']
        hourly_precipitation = data['hourly']['precipitation']
        
        monthly_temperatures = {str(month): [] for month in range(1, 13)}
        monthly_humidity = {str(month): [] for month in range(1, 13)}
        monthly_precipitation = {str(month): [] for month in range(1, 13)}
        
        for time, temp, hum, precip in zip(data['hourly']['time'], hourly_temperatures, hourly_humidity, hourly_precipitation):
            month = datetime.strptime(time, '%Y-%m-%dT%H:%M').month
            monthly_temperatures[str(month)].append(temp)
            monthly_humidity[str(month)].append(hum)
            monthly_precipitation[str(month)].append(precip)
        
        monthly_mean_temperatures = [round(sum(values) / len(values), 2) if values else None for values in monthly_temperatures.values()]
        monthly_mean_humidity = [round(sum(values) / len(values), 2) if values else None for values in monthly_humidity.values()]
        monthly_mean_precipitation = [round(sum(values) / len(values), 2) if values else None for values in monthly_precipitation.values()]
        
        return monthly_mean_temperatures, monthly_mean_humidity, monthly_mean_precipitation
    
    return None, None, None

# Read data from city_information.csv
df = pd.read_csv('city_information.csv')

# Initialize lists to store data
cities = []
latitudes = []
longitudes = []
monthly_mean_temperatures_list = []
monthly_mean_humidity_list = []
monthly_mean_precipitation_list = []

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    city = row['City']
    lat = row['Latitude']
    lon = row['Longitude']
    
    monthly_mean_temperatures, monthly_mean_humidity, monthly_mean_precipitation = extract_monthly_weather(lat, lon)
    
    if monthly_mean_temperatures is not None:
        cities.append(city)
        monthly_mean_temperatures_list.append(" ".join(map(str, monthly_mean_temperatures)))
        monthly_mean_humidity_list.append(" ".join(map(str, monthly_mean_humidity)))
        monthly_mean_precipitation_list.append(" ".join(map(str, monthly_mean_precipitation)))

# Create dataframe
result_df = pd.DataFrame({
    'City': cities,
    'Monthly Mean Temperature': monthly_mean_temperatures_list,
    'Monthly Mean Humidity': monthly_mean_humidity_list,
    'Monthly Mean Precipitation': monthly_mean_precipitation_list
})

# Write to CSV
result_df.to_csv('weather_data.csv', index=False)

print("Data has been written to weather_data.csv")


Data has been written to weather_data.csv


Collection of Air Quality parameters like PM10, PM2.5, Carbon Monoxide, Nitrogen Dioxide, Sulphur Dioxide, Ozone, Dust, UV Index & Ammonia on monthly average basis through API.

Every columns contains 12 space seperated values for each months.

In [3]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import time
import pandas as pd

# Define a function to calculate monthly averages
def calculate_monthly_averages(latitude, longitude):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://air-quality-api.open-meteo.com/v1/air-quality"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "dust", "uv_index", "ammonia"],
        "start_date": "2023-01-01",
        "end_date": "2023-12-31"
    }

    responses = openmeteo.weather_api(url, params=params)

    # Process hourly data for the first location
    response = responses[0]
    hourly = response.Hourly()
    hourly_data = {
        "pm10": hourly.Variables(0).ValuesAsNumpy(),
        "pm2_5": hourly.Variables(1).ValuesAsNumpy(),
        "carbon_monoxide": hourly.Variables(2).ValuesAsNumpy(),
        "nitrogen_dioxide": hourly.Variables(3).ValuesAsNumpy(),
        "sulphur_dioxide": hourly.Variables(4).ValuesAsNumpy(),
        "ozone": hourly.Variables(5).ValuesAsNumpy(),
        "dust": hourly.Variables(6).ValuesAsNumpy(),
        "uv_index": hourly.Variables(7).ValuesAsNumpy(),
        "ammonia": hourly.Variables(8).ValuesAsNumpy(),
        "date": pd.date_range(start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                              end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                              freq=pd.Timedelta(seconds=hourly.Interval()),
                              inclusive="left")
    }

    hourly_dataframe = pd.DataFrame(data=hourly_data)
    hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])
    hourly_dataframe['month'] = hourly_dataframe['date'].dt.month
    monthly_means = hourly_dataframe.groupby('month').mean(numeric_only=False)  # Specify numeric_only=False

    return monthly_means

# Read city information from CSV
city_info = pd.read_csv('city_information.csv', usecols=['City', 'Latitude', 'Longitude'])

# Create an empty DataFrame for output
output_df = pd.DataFrame(columns=['City', 'Monthly Average pm10', 'Monthly Average pm2.5',
                                  'Monthly Average carbon_monoxide', 'Monthly Average nitrogen_dioxide',
                                  'Monthly Average sulphur_dioxide', 'Monthly Average ozone',
                                  'Monthly Average dust', 'Monthly Average uv_index', 'Monthly Average ammonia'])

# Loop through each row of city information
for index, row in city_info.iterrows():
    city = row['City']
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Calculate monthly averages for the current city
    monthly_averages = calculate_monthly_averages(latitude, longitude)
    
    # Append monthly average values to the output DataFrame
    new_row = {'City': city,
           'Monthly Average pm10': ' '.join(map(str, monthly_averages['pm10'].tolist())),
           'Monthly Average pm2.5': ' '.join(map(str, monthly_averages['pm2_5'].tolist())),
           'Monthly Average carbon_monoxide': ' '.join(map(str, monthly_averages['carbon_monoxide'].tolist())),
           'Monthly Average nitrogen_dioxide': ' '.join(map(str, monthly_averages['nitrogen_dioxide'].tolist())),
           'Monthly Average sulphur_dioxide': ' '.join(map(str, monthly_averages['sulphur_dioxide'].tolist())),
           'Monthly Average ozone': ' '.join(map(str, monthly_averages['ozone'].tolist())),
           'Monthly Average dust': ' '.join(map(str, monthly_averages['dust'].tolist())),
           'Monthly Average uv_index': ' '.join(map(str, monthly_averages['uv_index'].tolist())),
           'Monthly Average ammonia': ' '.join(map(str, monthly_averages['ammonia'].tolist()))}
    time.sleep(1)
    print(f'Data extracted for {city}')

    # Concatenate the new row to the output DataFrame
    output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)

# Write output to CSV
output_df.to_csv('air_quality_data.csv', index=False)


Data extracted for The Hague
Data extracted for Eindhoven
Data extracted for Luxembourg
Data extracted for Rotterdam
Data extracted for Valencia
Data extracted for Helsinki
Data extracted for Amsterdam
Data extracted for Vienna
Data extracted for Munich
Data extracted for Copenhagen
Data extracted for Zurich
Data extracted for Reykjavik
Data extracted for Stuttgart
Data extracted for Geneva
Data extracted for Frankfurt
Data extracted for Porto
Data extracted for Madrid
Data extracted for Gothenburg
Data extracted for Tallinn
Data extracted for Cologne
Data extracted for Tokyo
Data extracted for Edinburgh
Data extracted for Muscat
Data extracted for Oslo
Data extracted for Glasgow
Data extracted for Split
Data extracted for Tampa
Data extracted for Austin
Data extracted for Cork
Data extracted for Hamburg
Data extracted for Charlotte
Data extracted for Brisbane
Data extracted for Orlando
Data extracted for Kansas City
Data extracted for Ljubljana
Data extracted for Dallas
Data extracted

Final task to merge 4 csv files to complete the datasets

In [4]:
# Read each CSV file
quality_of_life_df = pd.read_csv("modified_quality_of_life_indices.csv")
city_info_df = pd.read_csv("city_information.csv")
air_quality_df = pd.read_csv("air_quality_data.csv")
weather_df = pd.read_csv("weather_data.csv")

# Merge dataframes based on the 'City' column
merged_df = quality_of_life_df.merge(city_info_df, on='City') \
    .merge(air_quality_df, on='City') \
    .merge(weather_df, on='City')

# Write merged dataframe to a new CSV file
merged_df.to_csv("city_profile.csv", index=False)

print("Merged CSV file 'city_profile.csv' has been created successfully.")


Merged CSV file 'city_profile.csv' has been created successfully.
